In [9]:
import pennylane as qml
from pennylane import numpy as np
import autograd.numpy as np_grad
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score
import math

In [66]:
from pennylane.templates import AmplitudeEmbedding

dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def circuit_qml(feature=None, params=None):
    AmplitudeEmbedding(features=initial_state(feature), wires=range(2))
    for i in range(len(params)):
        qml.RX(params[i][0], wires=0)
        qml.RY(params[i][1], wires=0)
        qml.RX(params[i][2], wires=1)
        qml.RY(params[i][3], wires=1)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 0])
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

# activation function
def sigmoid(x):
    return 1 / (1 + np_grad.exp(-10*x))

def predict(feature, params):
    expval = circuit_qml(feature, params)
    pred = sigmoid(expval[0]-expval[1])
    return [pred, 1-pred]

# loss function    
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (1 - p[l]) ** 2
    loss = loss / len(labels)
    return loss

# loss function of QNN
def cost(params):
    features, labels = X_train, y_train
    preds = [predict(x, params) for x in features]
    return square_loss(labels, preds)

# encoding the features
def initial_state(feature):
    ampli_vec = np_grad.array([np_grad.sqrt(feature[0]), np_grad.sqrt(1-feature[0])])
    for i in range(1, len(feature)):
        ampli_vec = np_grad.kron(ampli_vec, np.array([np_grad.sqrt(feature[i]), np_grad.sqrt(1-feature[i])]))
    return ampli_vec

In [67]:
# source: https://www.kaggle.com/rakeshrau/social-network-ads
dataset = pd.read_csv('Social_Network_Ads.csv')
X = dataset.iloc[:, 2:-1].values
y = dataset.iloc[:, -1].values

# splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# scaling feature
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# create parameters  
n_layer = 3
params = np_grad.zeros((n_layer, 4,))
params

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [68]:
cost(params)

tensor(0.40831386, requires_grad=True)

In [70]:
opt = qml.AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-6)
steps = 200

start = time.time()
for i in range(steps):
    params = opt.step(cost, params)   
end = time.time()

In [72]:
cost(params)

tensor(0.09273073, requires_grad=True)

In [73]:
end-start

713.138840675354

In [75]:
pred = [predict(x, params) for x in X_train]
pred = np.argmax(pred, axis=1)

In [76]:
accuracy_score(y_train, pred)

0.890625